In [1]:
## Inputs
import dill as pickle
from datetime import datetime
from collections import OrderedDict
import time
with open(r"C:\Users\amusali\Desktop\uc3m PhD\05 Analysis\01 Main\00 Python data\03 Cleaned patents\all_patents.pkl", 'rb') as f:
    patents = pickle.load(f)

with open(r"C:\Users\amusali\Downloads\CheckedPatents_CLSonly_2024.11.11_00_06.pkl", 'rb') as f:
    cls_embeddings = pickle.load(f)

import pandas as pd 
## Convert patent citations into a dict
mydict = {patent.patent_id:{"cit":patent.forward_citations, "field": patent.tech_field_group_id, "date":patent.date_granted} for patent in patents}
merged ={}
for id, value in mydict.items():
    if id in cls_embeddings:
        merged[id] = {'citations':value['cit'], 'field': value['field'], "date":value['date'], 'cls': cls_embeddings[id]}
print("Here")
i=0
for key, value in merged.items():
    print(key, value)
    i+=1
    if i>10:
        break
cls_emmbedings = None
#df = pd.DataFrame()

Here
5126739 {'citations': 196, 'field': 'G06T', 'date': datetime.datetime(1992, 6, 30, 0, 0), 'cls': array([-0.28444612, -1.304903  , -0.27453208, ..., -0.07086144,
        0.12113389,  0.06876312], dtype=float32)}
5414850 {'citations': 129, 'field': 'G11B', 'date': datetime.datetime(1995, 5, 9, 0, 0), 'cls': array([[-0.43464324, -0.5421706 , -0.3452945 , ..., -0.15224704,
         0.41279787,  0.29669955]], dtype=float32)}
5442708 {'citations': 64, 'field': 'H04L', 'date': datetime.datetime(1995, 8, 15, 0, 0), 'cls': array([ 0.21176712, -1.2779055 , -0.6300596 , ..., -0.4678112 ,
       -0.01390232, -0.34096363], dtype=float32)}
5444782 {'citations': 131, 'field': 'H04L', 'date': datetime.datetime(1995, 8, 22, 0, 0), 'cls': array([ 0.43182442, -1.5171539 , -0.43305537, ..., -0.5866163 ,
       -0.01163151, -0.7051122 ], dtype=float32)}
5537586 {'citations': 323, 'field': 'G06F', 'date': datetime.datetime(1996, 7, 16, 0, 0), 'cls': array([-1.4246577e+00, -1.2845162e+00,  2.4936986e-01

In [7]:
import pandas as pd
import numpy as np
from sklearn.linear_model import LassoCV
from sklearn.model_selection import cross_val_score

# Example dictionary with patents and their citations/CLS embeddings
data = merged
# Step 1: Convert Dictionary to DataFrame
records = []
for patent_id, values in data.items():

    row = {'patent_id': patent_id, 'citations': values['citations'], 'field': values['field'], 'date':values['date']}
    # Flatten the CLS embeddings
    cls_flat = values['cls'].flatten()
    row.update({f'cls{i+1}': val for i, val in enumerate(cls_flat)})
    records.append(row)

# Create DataFrame
df = pd.DataFrame(records)

# Step 2: Prepare Features (CLS embeddings) and Target (Citations)
X = df.drop(columns=['patent_id', 'citations']).values  # Features: cls1, cls2, ..., clsN
y = df['citations'].values  # Target: citations
y = np.round(y).astype(int)  # Ensure y is rounded and an integer
print(np.isnan(X).any(), np.isnan(y).any())
print(len(X), len(y))



TypeError: ufunc 'isnan' not supported for the input types, and the inputs could not be safely coerced to any supported types according to the casting rule ''safe''

In [8]:
df.head(5)

,patent_id,citations,field,date,cls1,cls2,cls3,cls4,cls5,cls6,...,cls1015,cls1016,cls1017,cls1018,cls1019,cls1020,cls1021,cls1022,cls1023,cls1024
0,5126739,196,G06T,1992-06-30,-0.284446,-1.304903,-0.274532,1.131838,0.121896,-0.399315,...,0.586855,0.543201,-0.324679,0.965743,-0.006917,0.280171,1.752606,-0.070861,0.121134,0.068763
1,5414850,129,G11B,1995-05-09,-0.434643,-0.542171,-0.345295,1.728004,-0.246377,-0.050537,...,0.055327,0.721785,0.063163,1.038806,-0.787815,-0.527328,0.393609,-0.152247,0.412798,0.296700
2,5442708,64,H04L,1995-08-15,0.211767,-1.277905,-0.630060,1.178841,-0.059403,0.156697,...,0.289955,1.772817,-0.770613,0.377784,0.755178,-0.450753,0.016557,-0.467811,-0.013902,-0.340964
3,5444782,131,H04L,1995-08-22,0.431824,-1.517154,-0.433055,0.536814,0.087209,-0.022103,...,0.084899,1.516218,-0.672385,0.085915,0.538845,-0.305390,0.123248,-0.586616,-0.011632,-0.705112
4,5537586,323,G06F,1996-07-16,-1.424658,-1.284516,0.249370,0.347558,-0.630628,0.413755,...,0.933559,0.790688,0.810933,-0.173193,-0.853715,-0.213430,1.447543,-1.097498,0.308361,-0.000822


In [13]:

# Step 3: Fit Lasso Regression with 5-Fold Cross-Validation
lasso = LassoCV(cv=5, max_iter=100000)
lasso.fit(X, y)

# Step 4: Print Results
print("Optimal Alpha (Regularization Strength):", lasso.alpha_)
print("Coefficients (CLS Embeddings):")
print(pd.Series(lasso.coef_, index=[f'cls{i+1}' for i in range(X.shape[1])]))

# Cross-validation scores
cv_scores = cross_val_score(lasso, X, y, cv=5, scoring='r2')
print("R2 Scores Across Folds:", cv_scores)
print("Mean R2 Score:", np.mean(cv_scores))

Optimal Alpha (Regularization Strength): 0.5329084839409096
Coefficients (CLS Embeddings):
cls1      -0.000000
cls2       0.000000
cls3      -0.000000
cls4       4.462394
cls5      -0.324580
             ...   
cls1020   -0.000000
cls1021   -0.000000
cls1022    0.000000
cls1023   -0.000000
cls1024    0.000000
Length: 1024, dtype: float32
R2 Scores Across Folds: [-0.06383884 -0.05461836 -0.50290549 -0.18456089 -0.56497145]
Mean R2 Score: -0.2741790056228638


In [16]:
# After fitting the Lasso model
coefficients = lasso.coef_

# Count non-zero and zero coefficients
nonzero_count = np.sum(coefficients != 0)
zero_count = np.sum(coefficients == 0)

# Print results
print(f"Number of features with non-zero coefficients: {nonzero_count}")
print(f"Number of features with zero coefficients: {zero_count}")


Number of features with non-zero coefficients: 173
Number of features with zero coefficients: 851


In [17]:
from sklearn.ensemble import RandomForestRegressor
from sklearn.model_selection import cross_val_score

rf = RandomForestRegressor(n_estimators=100, random_state=42)
scores = cross_val_score(rf, X, y, cv=5, scoring='r2')
print("Random Forest R2 Scores:", scores)
print("Mean R2 Score:", np.mean(scores))


KeyboardInterrupt: 

In [44]:
import numpy as np
import pandas as pd
from sklearn.linear_model import LinearRegression
from sklearn.model_selection import cross_val_score
import random

# Assuming 'df' is your DataFrame with 'citations' as the target column
# and 'cls1' to 'cls1024' as feature columns.

# Step 1: Extract feature columns and target
feature_columns = [col for col in df.columns if col.startswith('cls')]
target = df['citations']

# Step 2: Randomly select 50 features
selected_features = random.sample(feature_columns, 1024)

# Step 3: Prepare the data for OLS
X_selected = df[selected_features].values
y = target.values

# Step 4: Fit OLS model (Linear Regression)
ols = LinearRegression()

# Step 5: Cross-validate the model using 5-fold CV
cv_scores = cross_val_score(ols, X_selected, y, cv=5, scoring='r2')

# Print Results
print("Randomly Selected Features (50):", selected_features)
print("R2 Scores Across Folds:", cv_scores)
print("Mean R2 Score:", np.mean(cv_scores))


Randomly Selected Features (50): ['cls174', 'cls953', 'cls958', 'cls191', 'cls446', 'cls993', 'cls941', 'cls193', 'cls275', 'cls8', 'cls522', 'cls555', 'cls35', 'cls609', 'cls925', 'cls123', 'cls265', 'cls16', 'cls659', 'cls992', 'cls524', 'cls943', 'cls102', 'cls981', 'cls405', 'cls377', 'cls186', 'cls500', 'cls439', 'cls297', 'cls326', 'cls733', 'cls863', 'cls615', 'cls228', 'cls31', 'cls933', 'cls896', 'cls700', 'cls306', 'cls962', 'cls425', 'cls848', 'cls586', 'cls499', 'cls169', 'cls581', 'cls858', 'cls143', 'cls779', 'cls538', 'cls211', 'cls732', 'cls789', 'cls37', 'cls292', 'cls491', 'cls194', 'cls846', 'cls826', 'cls72', 'cls300', 'cls460', 'cls682', 'cls959', 'cls109', 'cls61', 'cls911', 'cls294', 'cls3', 'cls305', 'cls141', 'cls481', 'cls937', 'cls21', 'cls83', 'cls386', 'cls999', 'cls398', 'cls922', 'cls866', 'cls348', 'cls15', 'cls375', 'cls734', 'cls200', 'cls316', 'cls669', 'cls362', 'cls622', 'cls830', 'cls74', 'cls626', 'cls655', 'cls91', 'cls236', 'cls154', 'cls112', '

In [10]:
import pandas as pd

# Assuming 'df' is your DataFrame
output_file = "patent_data.xlsx"

# Export DataFrame to Excel
df.to_excel(output_file, index=False, sheet_name="CLS_Embeddings")

print(f"DataFrame has been exported to '{output_file}' successfully!")


DataFrame has been exported to 'patent_data.xlsx' successfully!
